<a href="https://colab.research.google.com/github/Jooseok/Test/blob/main/HF_TF_to_TFLite_transcribe_fails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00


In [ ]:
%%capture
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets

In [ ]:
import transformers
print(transformers.__version__)


4.35.0.dev0


In [ ]:
from transformers import WhisperProcessor, TFWhisperForConditionalGeneration, GenerationConfig
from datasets import Audio, load_dataset
import tensorflow as tf
import numpy as np
from transformers import TFForceTokensLogitsProcessor, TFLogitsProcessor
from typing import List, Optional, Union, Any

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny")
model = TFWhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="es", task="translate")
#forced_decoder_ids = processor.get_decoder_prompt_ids(task="transcribe")

# load streaming dataset and read first audio sample
ds = load_dataset("common_voice", "es", split="test", streaming=True)
ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
input_speech = next(iter(ds))["audio"]
input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="tf").input_features
model.config.forced_decoder_ids = forced_decoder_ids
generation_config = GenerationConfig(force_token_map=forced_decoder_ids)
print(generation_config)
# generate token ids


# Patching methods of class TFForceTokensLogitsProcessor(TFLogitsProcessor):

def my__init__(self, force_token_map: List[List[int]]):
    force_token_map = dict(force_token_map)
    # Converts the dictionary of format {index: token} containing the tokens to be forced to an array, where the
    # index of the array corresponds to the index of the token to be forced, for XLA compatibility.
    # Indexes without forced tokens will have an negative value.
    force_token_array = np.ones((max(force_token_map.keys()) + 1), dtype=np.int32) * -1
    for index, token in force_token_map.items():
        if token is not None:
            force_token_array[index] = token
    self.force_token_array = tf.convert_to_tensor(force_token_array, dtype=tf.int32)

def my__call__(self, input_ids: tf.Tensor, scores: tf.Tensor, cur_len: int) -> tf.Tensor:
    def _force_token(generation_idx):
        batch_size = scores.shape[0]
        current_token = self.force_token_array[generation_idx]

        # Original code below generates NaN values when the model is exported to tflite
        # it just needs to be a negative number so that the forced token's value of 0 is the largest
        # so it will get chosen
        #new_scores = tf.ones_like(scores, dtype=scores.dtype) * -float("inf")
        new_scores = tf.ones_like(scores, dtype=scores.dtype) * -float(1)
        indices = tf.stack((tf.range(batch_size), tf.tile([current_token], [batch_size])), axis=1)
        updates = tf.zeros((batch_size,), dtype=scores.dtype)
        new_scores = tf.tensor_scatter_nd_update(new_scores, indices, updates)
        return new_scores

    scores = tf.cond(
        tf.greater_equal(cur_len, tf.shape(self.force_token_array)[0]),
        # If the current length is geq than the length of force_token_array, the processor does nothing.
        lambda: tf.identity(scores),
        # Otherwise, it may force a certain token.
        lambda: tf.cond(
            tf.greater_equal(self.force_token_array[cur_len], 0),
            # Only valid (positive) tokens are forced
            lambda: _force_token(cur_len),
            # Otherwise, the processor does nothing.
            lambda: scores,
        ),
    )
    return scores

TFForceTokensLogitsProcessor.__init__ = my__init__
TFForceTokensLogitsProcessor.__call__ = my__call__
predicted_ids = model.generate(input_features,generation_config=generation_config)
print(predicted_ids)
# decode token ids to text
transcription = processor.batch_decode(predicted_ids)

transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)

All PyTorch model weights were used when initializing TFWhisperForConditionalGeneration.

All the weights of TFWhisperForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFWhisperForConditionalGeneration for predictions without further training.
/root/.cache/huggingface/modules/datasets_modules/datasets/common_voice/220833898d6a60c50f621126e51fb22eb2dfe5244392c70dccd8e6e2f055f4bf/common_voice.py:634: FutureWarning: 
            This version of the Common Voice dataset is deprecated.
            You can download the latest one with
            >>> load_dataset("mozilla-foundation/common_voice_11_0", "en")
            
  warnings.warn(


GenerationConfig {
  "force_token_map": [
    [
      1,
      50262
    ],
    [
      2,
      50358
    ],
    [
      3,
      50363
    ]
  ]
}



/usr/local/lib/python3.10/dist-packages/transformers/generation/tf_utils.py:838: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tf.Tensor(
[[50258 50262 50358 50363   731   293   286   486   747   309   484   700
  50257]], shape=(1, 13), dtype=int32)
[' well and I will take it out first']


In [ ]:
model.save('/content/tf_whisper_saved')

In [ ]:
import tensorflow as tf
class GenerateModel(tf.Module):
  def __init__(self, model):
    super(GenerateModel, self).__init__()
    self.model = model

  @tf.function(
    input_signature=[
      tf.TensorSpec(shape=(1, 80,3000), dtype=tf.float32, name="input_ids"),
    ]
  )
  def serving(self, input_ids):
    outputs = self.model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    return {"sequences": outputs}

# Saving the model
saved_model_dir = '/content/tf_whisper_saved'
generate_model = GenerateModel(model=model)
tf.saved_model.save(generate_model, saved_model_dir, signatures={"serving_default": generate_model.serving})

# Converting to TFLite model
tflite_model_path = '/content/whisper-tiny.tflite'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Saving the TFLite model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [ ]:
%ls -la

total 150260
drwxr-xr-x 1 root root      4096 Oct 24 03:07 ./
drwxr-xr-x 1 root root      4096 Oct 24 03:02 ../
drwxr-xr-x 4 root root      4096 Oct 20 13:20 .config/
drwxr-xr-x 1 root root      4096 Oct 20 13:20 sample_data/
drwxr-xr-x 4 root root      4096 Oct 24 03:33 tf_whisper_saved/
-rw-r--r-- 1 root root 153845620 Oct 24 03:34 whisper-tiny.tflite


In [ ]:
!pip install whisper

In [ ]:
# Import necessary libraries
import whisper
import numpy as np
from timeit import default_timer as timer


# Define the path to the TFLite model
tflite_model_path = '/content/whisper-tiny.tflite'

# Create an interpreter to run the TFLite model
interpreter = tf.lite.Interpreter(tflite_model_path)

# Allocate memory for the interpreter
interpreter.allocate_tensors()

# Get the input and output tensors
input_tensor = interpreter.get_input_details()[0]['index']
output_tensor = interpreter.get_output_details()[0]['index']


inference_start = timer()

# Run the TFLite model using the interpreter
print("Invoking interpreter ...")
interpreter.set_tensor(input_tensor, input_features)
interpreter.invoke()

# Get the output data from the interpreter
output_data = interpreter.get_tensor(output_tensor)

# Print the output data
print(output_data)
transcription = processor.batch_decode(output_data, skip_special_tokens=True)
print(transcription)
# Create a tokenizer to convert tokens to text
#wtokenizer = whisper.tokenizer.get_tokenizer(True, language="fr")

# convert tokens to text
#print("Converting tokens ...")
#for token in output_data:
    # Replace -100 with the end of text token
  #  token[token == -100] = wtokenizer.eot
    #text = wtokenizer.decode(token, skip_special_tokens=True)
  #  print(text)

print("\nInference took {:.2f}s ".format(timer() - inference_start))

Invoking interpreter ...
[[50258 50262 50358 50363   731   293   286   486   747   309   484   700
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257 50257
  50257 50257